<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/Transformer_%EA%B5%AC%ED%98%84_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 Transformer model 구현

Transformer의 기본 동작 원리만을 구현한 것

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.function as F   # old version의 Ptyorch에 있는 함수라 선언 불가가

import copy

class Transformer(nn.Module):
  def __init__(self, encoder, decoder):
    super(Transformer, self).__init__()
    self.encoder = encoder
    self.decoder = decoder

  def encode(self, src, src_mask):
    return self.encoder(self.src_embed(src), src_mask)

  def decode(self, tgt, encoder_out, tgt_mask, src_tgt_mask):
    return self.decoder(self.tgt_embed(tgt), encoder_out, tgt_mask, src_tgt_mask)

  def forward(self, src, tgt, src_mask):
    # (추가) mask
    # (추가) generator
    src_mask = self.make_src_mask(src)
    tgt_mask = self.make_tgt_mask(tgt)
    src_tgt_mask = self.make_src_tgt_mask(src, tgt)
    encoder_out = self.encode(src, src_mask)
    decoder_out = self.decode(tgt, encoder_out, tgt_mask, src_tgt_mask)
    out = self.generator(decoder_out)
    out = F.log_softmax(out, dim = -1)
    return out, decoder_out

  def make_src_mask(self, src):
    pad_mask = self.make_pad_mask(src, src)
    return pad_mask

ModuleNotFoundError: ignored

## Encoder block 구현
Encoder는 sentence에서 context를 파악하는 역할. 반복되는 Encoder block을 통해 이전의 Encoder block의 출력을 다음의 Encoder block이 사용하게 되면서, 더욱 세밀하게 context 파악. 

In [7]:
class Encoder(nn.Module):

  def __init__(self, encoder_block, n_layer):   # n_layer: 블록의 갯수
    super(Encoder, self).__init__()
    self.layers = []
    for i in range(n_layer):
      self.layers.append(copy.deepcopy(encoder_block))

  def forward(self, src, src_mask):   # 이전 Encoder block의 output을 현재 Encoder block의 input으로 사용용
    out = src
    for layer in self.layers:
      out = layer(out, src_mask)
    return out

Encoder block은 크게 Multi-Head Attention Layer와 Position-wise Feed-FOrward Layer로 구성된다.

In [8]:
class EncoderBlock(nn.Module):

  def __init__(self, self_attention, position_ff):
    super(EncoderBlock, self).__init__()
    self.self_attention = self_attention   # Multi-Head Attention
    self.position_ff = position_ff         # Position-wise Feed-Forward
    self.residuals = [ResidualConnectionLayer() for _ in range(2)]   # (추가) Reisdual connection

  def forward(self, src, src_mask):
    out = src
    out = self.residuals[0](out, lambda out: self.self_attention(query = out, key = out, value = out, mask = src_mask))
    out = self.residuals[1](out, self.position_ff(out))
    return out

### Attention이 무엇일까?

Multi-Head Attention은 Scaled DOt-Product Attention을 병렬적으로 수행하는 layer이다. 이렇게만 설명하면 이해를 하기 힘든데, 우선 Attention은 넓은 범위의 전체 data에서 특정한 부분에 집중한다는 의미이다. 그리고 이 Scaled-Dot Product Attention을 줄여서 Attention이라고 부르기도 한다. 이러한 Attention 중에서도 **같은 문장 내**의 두 토큰 사이의 Attention을 계산하는 방법을 *Self-Attention*이라고 한다.

### Query, Key, Value

이제 Attention이 어떤 방식으로 작동하는지 살펴보자. Attention 계산에는 Query, Key, Value 이렇게 3가지의 벡터가 사용되는데, 각 벡터의 역할은 다음과 같다.

1. Query: 현 시점의 token
2. Key: attention 값을 구하고자 하는 대상 token
3. Value: attention 값을 구하고자 하는 대상 token (Key와 동일한 token)

Key와 Value는 완전히 동일한 값을 가지는데, 이 둘은 문장의 처음부터 끝까지 탐색한다. Query는 고정된 하나의 값을 가지는데, Query와 가장 부합하는 token을 찾기 위해 Key, Value를 문장의 처음부터 끝까지 탐색시키는 것이다.

### Scaled Dot-Product Attention

이 Scaled Dot-Product Attention의 작동 방식에 대한 자세한 내용은 블로그를 참고하길 바란다. https://cartinoe5930.tistory.com/entry/Transformer-Attention-Is-All-You-Need-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0



이러한 Self-Attention을 pytorch code로 구현하면 다음과 같다. 

In [9]:
def calculate_attention(query, key, value, mask):
  # query, key, value: (n_batch, seq_len, d_k)
  # mask: (n_batch, seq_len, seq_len)
  d_k = key.shape[-1]
  attention_score = torch.matmul(query, key.transpose(-2, -1)) # Q x K^T, (n_batch, seq_len, seq_len)
  attention_score = attention_score / math.sqrt(d_k) # Scale
  if mask is not None: # Mask(Opt.)
    attention_score = attention_score.masked_fill(mask == 0, -1e9)
  attention_prob = F.softmax(attention_score, dim = -1) # SoftMax (n_batch, seq_len, seq_len)
  out = torch.matmul(attention_prob, value) # last MatMul
  return out

ModuleNotFoundError: ignored

### Multi-Head Attention

지금까지의 Self-Attention 개념은 모두 Multi-Head Attention을 이해하기 위한 것이었다. Transformer의 Encoder layer에서 Scaled Dot-Product Attention은 1회씩 수행하는 것이 아니라 병렬적으로 h회 수행한 뒤에, 그 결과를 종합해서 사용한다. 이러한 연산을 수행하는 이유는 다양한 Attention을 잘 반영하기 위해서이다. 논문에서는 이 h를 8로 설정하였다. 자세한 내용은 블로그를 참고하길 바란다.

https://cartinoe5930.tistory.com/entry/Transformer-Attention-Is-All-You-Need-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0

이 Multi-Head Attention을 pytorch code로 구현하면 다음과 같다.

In [ ]:
class MultiHeadAttentionLayer(nn.Module):

  def __init__(self, d_model, h, qkv_fc, out_fc):
    super(MultiHeadAttentionLayer, self).__init__()
    self.d_model = d_model
    self.h = h
    self.q_fc = copy.deepcopy(qkv_fc)
    self.k_fc = copy.deepcopy(qkv_fc)
    self.v_fc = copy.deepcopy(qkv_fc)
    self.out_fc = out_fc

  def forward(self, *arge, query, key, value, mask = None):   # Transformer에서 가장 중요한 부분이니 확실히 이해하자!
    n_batch = query.size(0)

    def transform(x, fc):   # (n_batch, seq_len, d_embed)
      out = fc(x)           # (n_batch, seq_len, d_model)
      out = out.view(n_batch, -1, self.h, self.d_model, self.d_model//self.h)   # (n_batch, seq_len, h, d_k)
      out = out.transpose(1, 2)   # (n_batch, h, seq_len, d_k)
      return out

    query = transform(query, self.q_fc)
    key = transform(key, self.k_fc)
    value = transform(value, self.v_fc)

    out = self.calculate_attention(query, key, value, mask)   # Self-Attention (n_batch, h, seq_len, d_k)
    out = out.transpose(1, 2)   # (n_batch, seq_len, h, d_k)
    out = out.contiguous().view(n_batch, -1, self.d_model)   # (n_batch, seq_len, d_model)
    out = self.out_fc(out)   # (n_batch, seq_len, d_model)
    return out

인자로 받고 있는 Query, Key, Value는 아직 실제 Q, K, V 행렬이 아닌, input sentence embedding이다. 이를 3개의 FC layer에 넣어서 각각의 Q, K, V를 구하는 것이다.

transform()는 Q, K, V를 구하는 함수이다. 따라서 input shape는 (n_batch x seq_len x d_embed)이고, output shape는 (n_batch x seq_len x d_model)이다. 하지만, 이 중간에 d_model을 h와 d_k로 분리해서, shape는 (n_batch x seq_len x h x d_k)가 된다. 그리고 transpose를 통해 (n_batch x h x seq_len x d_k)로 변형하는데, 이는 calculate_attention()의 입력으로 사용해야 하기 때문이다. 

다시 forward()로 돌아와서, calculate_attention을 사용해서 attention을 계산하면 그 shape이 (n_batch x h x seq_len x d_k)이다. 하지만, Multi-Head Attention도 shape에 대해 멱등해야 하기 때문에, output shape는 input과 같은 (n_batch x seq_len x d_embed)여야 한다. 이를 위해 transpose를 한 후, d_k와 h를 결합하여 d_model을 만든다. 이후에 FC layer을 거쳐서 d_model을 d_embed로 변환한다.

그리고 mask 인자를 받기 위해 Encoder block과 Transformer도 수정해야 한다.

### Pad Mask

그동안 생략했던 과정인 pad masking을 생성하는 make_pad_mask()이다. 여기서 pad의 인덱스를 의미하는 pad_idx와 일치하는 token들은 모두 0, 그 외에는 모두 1인 mask를 생성한다.

In [ ]:
def make_pad_mask(self, query, key, pad_idx = 1):
  query_seq_len, key_seq_len = query.size(1), key.size(1)
  
  key_mask = key.ne(pad_idx).unsqueeze(1).unsqueeze(2)   # (n_batch, 1, 1, key_seq_len)
  key_mask = key_mask.repeat(1, 1, query_seq_len, 1)     # (n_batch, 1, query_seq_len, key_seq_len)

  query_mask = query.ne(pad_idx).unsqueeze(1).unsqueeze(2)   # (n_batch, 1, 1, key_seq_len)
  query_mask = query_mask.repeat(1, 1, 1, key_seq_len)       # (n_batch, 1, query_seq_len, key_seq_len)

  mask = key_mask & query_mask
  mask.requires_grad = False
  return mask

pad mask는 개념적으로 Encoder 내부에서 생성하는 것이 아니기 때문에, Transformer의 method로 위치시킨다.

### Position-wise Feed Forward

단순하게 2개의 FC layer를 갖는 layer이다. 각 FC layer는 (d_embed x d_ff), (d_ff x d_embed)의 가중치 행렬을 갖는다. 즉, Feed Forward Layer 역시 shape에 대해 멱등하다. 다음 Encoder Block에게 shape를 유지할 채 넘겨줘야 하기 때문이다. 정리하면, Feed Forward layer는 Multi-Head Attention layer의 output을 입력으로 받아 연산을 수행하고, 다음 Encoder Block에게 output을 넘겨준다. 

Position-wise Feed Forward를 pytorch code로 구현하면 다음과 같다.

In [ ]:
class PositionWiseFeedForwardLayer(nn.Module):

  def __init__(self, fc1, fc2):
    super(PositionWiseFeedForwardLayer, self).__init__()
    self.fc1 = fc1   # (d_embed, d_ff)
    self.relu = nn.ReLU()
    self.fc2 = fc2   # (d_ff, d_embed)
  
  def forward(self, x):
    out = x
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

### Residual Connection Layer

Encoder block은 Multi-Head Attention layer와 Position-wise Feed-Forward layer로 구성되어있다. 그러나 사실은 Encoder Block을 구성하는 두 layer는 Residual Connection으로 둘러싸여 있다. 

이를 pytorch code로 구현하면 다음과 같다. 정말 간단한다!

In [ ]:
class ResidualConnectionLayer(nn.Module):

  def __init__(self):
    super(ResidualConnectionLayer, self).__init__()

  def forward(self, x, sub_layer):
    out = x
    out = sub_layer(out)
    out = out + x   # f(x) = f(x) + x
    return out

따라서 Encoder Block의 code가 변경되게 된다.

## Decoder 구현

Decoder는 Encdoer의 출력인 context와 추가적인 sentence를 input으로 받아 output sentence를 출력한다. 여기서 의문이 생길 것이다. context를 받는 것 까지는 이해가 가는데, sentence는 무엇일까?

#### Teacher forcing

Decoder에 추가적으로 들어오는 sentence를 이해하기 위해서는 Teacher Forcing이라는 개념을 알아야 한다. Teacher Forcing은 지도 학습에서, label data를 input으로 사용하는 것이다. 이를 통해서 model이 잘못된 token을 생성하더라도 이후에 제대로 된 token을 생성해내도록 유도할 수 있는 것이다.

따라서 이제, input으로 들어오는 sentence가 ground_truth[:-1]의 sentence라는 것을 알 수 있다. 하지만, Transformer는 RNN과 달리 이전 cell의 값을 참고할 수 없다. 왜냐하면 Transformer는 병렬 연산을 하기 때문이다. 따라서, masking을 적용해야 한다. 이러한 masking을 subsequent masking이라 한다. 이를 pytorch code로 구현하면 다음과 같다.

In [ ]:
def make_subsequent_mask(query, key):
  query_seq_len, key_seq_len = query.size(1), key.size(1)

  tril = np.tril(np.ones((query_seq_len, key_seq_len)), k = 0).astype('uint8')
  mask = torch.tensor(tril, dtype = torch.bool, requires_grad = False, device = query.device)
  return mask

make_subsequent_mask()는 np.tril()을 사용하여 lower triangle을 생성한다. 

이를 통해 나온 결과는, n 번째 토큰은 0~n-1 번째 토큰을 참고할 수 있다.

하지만, 동시에 Encoder와 마찬가지로 pad masking 역시 적용되어야 하기 때문에, make_tgt_mask()는 다음과 같다. make_subsequent_mask(), make_tgt_mask(), make_src_mask()는 Transformer의 method로 작성한다.

In [ ]:
def make_tgt_mask(self, tgt):
  pad_mask = self.make_pad_mask(tgt, tgt)
  seq_mask = self.make_subsequent_mask(tgt, tgt)
  mask = pad_mask & seq_mask
  return pad_mask & seq_mask

Transformer로 다시 돌아가서, 기존에는 pad_mask만이 forward()를 구해야 했다면, 이제는 Decoder에서 사용할 subsequent + pad_mask도 구해야 한다. forward() 내부에서 Decoder의 forward()를 호줄할 때 역시 변경되는데, tgt_mask가 추가적으로 인자로 넘어가게 된다.

### Decoder Block

Decoder 역시 Encoder와 마찬가지로 N개의 Decoder Block이 겹겹이 쌓여있는 구조이다. 이때, Encoder에서 넘어오는 context가 각 Decoder Block마다 input으로 주어진다는 것이다. 그 외에는 Encoder와 차이가 전혀 없다.

Decoder의 attention layer는 총 두 가지로 이루어져 있는데, 하나는 Masked-Multi-Head Attention layer이다. 이렇게 불리는 이유는 pad masking 뿐만 아니라 subsequent masking까지 적용되기 때문이다. 다른 하나는 Encoder에서 넘어온 context를 key와 value로 사용하기 때문에, Cross-Multi-Head Attention이다. 

더욱 자세한 내용은 블로그를 참고하길 바란다.

https://cartinoe5930.tistory.com/entry/Transformer-Attention-Is-All-You-Need-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0

In [ ]:
class Decoder(nn.Module):
  
  def __init__(self, decoder_block, n_layer):
    super(Decoder, self).__init__()
    self.n_layer = n_layer
    self.layers = nn.ModuleList([copy.deepcopy(decoder_block) for _ in range(self.n_layer)])

  def forward(self, tgt, encdoer_out, tgt_mask, src_tgt_mask):
    # encoder_out: context로부터 얻어낸 key와 value
    # tgt_mask: make_tgt_mask()로 얻어진 mask
    out = tgt
    for layer in self.layers:
      out = layer(out, encdoer_out, tgt_mask, src_tgt_mask)
    return out

Decoder Block은 Encoder Block과 별 차이 없다. forward()에서 self_attention과 달리 cross_attention의 key, value는 encoder_out이라는 것, 각각 mask가 tgt_mask, src_tgt_mask라는 것만 주의하면 된다.

In [ ]:
class DecoderBlock(nn.Module):

  def __init__(self, self_attention, cross_attention, position_ff):
    super(DecoderBlock, self).__init__()
    self.self_attention = self_attention
    self.cross_attention = cross_attention
    self.position_ff = position_ff
    self.residuals = [ResidualConnectionLayer() for _ in range(3)]

  def forward(self, tgt, encoder_out, tgt_mask, src_tgt_mask):
    out = tgt
    out = self.residuals[0](out, lambda out: self.self_attention(query = out, key = out, value = out, mask = tgt_mask))
    out = self.residuals[1](out, lambda out: self.cross_attention(query = out, key = encoder_out, value = encoder_out, mask = src_tgt_mask))
    out = self.residuals[2](out, self.position_ff)
    return out

이에 따라 Transformer도 src_tgt_mask를 포함해 수정된다.

## Transformer Input (Positional Embedding)

Transformer의 Embedding은 단순하게 Token Embedding과 Positional Encoding의 sequential로 구성된다.

In [ ]:
class TransformerEmbedding(nn.Module):

  def __init__(self, otken_embed, pos_embed):
    super(TransformerEmbedding, self).__init__()
    self.embedding = nn.Sequential(token_embed, pos_embed)

  def forward(self, x):
    out = self.embedding(x)
    return out

Token Embedding 또한 역시 단순한다. vocabulary와 d_embed를 사용해서 embedding을 생성한다.

In [ ]:
class TokenEmbedding(nn.Module):
  
  def __init__(self, d_embed, vocab_size):
    super(TokenEmbedding, self).__init__()
    self.embedding = nn.Embedding(vocab_size, d_embed)
    self.d_embed = d_embed

  def forward(self, x):
    out = self.embedding(x) * math.sqrt(self.d_embed)
    return out

마지막으로 Positional Encoding을 살펴보자.

In [ ]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_embed, max_len = 256, device = torch.device('cpu')):
    super(PositionalEncoding, self).__init__()
    encoding = torch.zeros(max_len, d_embed)
    encoding.requires_grad = False
    position = torch.arange(0, max_len).float().unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed))
    encoding[:, 0::2] = torch.sin(position * div_term)
    encoding[:, 1::2] = torch.cos(position * div_term)
    self.encoding = encoding.unsqueeze(0).to(device)

  def forward(self, x):
    _, seq_len, _ = x.size()
    pos_embed = self.encdoing[:, :seq_len, :]
    out = x + pos_embed
    return out

Positional Encoding의 목적은 positional 정보를 정규화시키기 위한 것이다. positional 정보를 일정한 범위 안의 실수로 제약해두는데, 여기서 sin함수와 cos함수를 사용한다. 짝수 index에는 sin함수를, 홀수 index에는 cos함수를 사용한다. 이를 사용할 경우 항상 -1에서 1 사이의 값만이 positional 정보로 사용되게 된다.

이렇게 생성해낸 embedding을 Transformer에 추가해주자.

### Generator

Decoder의 output이 그대로 Transformer의 최종 output이 되는 것은 아니다. 즉, Embedding이 아닌 실제 target vocab 에서의 token sequence를 원하는 것이다. 이를 위해 추가적인 FC layer을 거치는데 이를 대게 Generator라고 부른다.

Generator가 하는 일은 Decoder output의 마지막 dimension을 d_embed에서 len(vocab)으로 변경하는 것이다. 이를 통해 실제 vocabulary 내 token에 대응시킬 수 있는 shape가 된다. 이후에 softmax()를 사용해 각 covabulary에 대한 확률값으로 변환한다.

Generator을 Transformer에 추가시켜 보자.

## Factory Method

Transformer를 생성하는 build_model()은 다음과 같이 작성할 수 있다. 각 module의 submodule을 생성자 내부에서 생성하지 않고, 외부에서 인자로 받는 이유는 더 자유롭게 모델을 변경해 응용할 수 있게 하기 위함이다.

In [ ]:
def build_model(src_vocab_size, tgt_vocab_size, device=torch.device("cpu"), max_len=256, d_embed=512, n_layer=6, d_model=512, h=8, d_ff=2048):
    import copy
    copy = copy.deepcopy

    src_token_embed = TokenEmbedding(
                                     d_embed = d_embed,
                                     vocab_size = src_vocab_size)
    tgt_token_embed = TokenEmbedding(
                                     d_embed = d_embed,
                                     vocab_size = tgt_vocab_size)
    pos_embed = PositionalEncoding(
                                   d_embed = d_embed,
                                   max_len = max_len,
                                   device = device)

    src_embed = TransformerEmbedding(
                                     token_embed = src_token_embed,
                                     pos_embed = copy(pos_embed))
    tgt_embed = TransformerEmbedding(
                                     token_embed = tgt_token_embed,
                                     pos_embed = copy(pos_embed))

    attention = MultiHeadAttentionLayer(
                                        d_model = d_model,
                                        h = h,
                                        qkv_fc = nn.Linear(d_embed, d_model),
                                        out_fc = nn.Linear(d_model, d_embed))
    position_ff = PositionWiseFeedForwardLayer(
                                               fc1 = nn.Linear(d_embed, d_ff),
                                               fc2 = nn.Linear(d_ff, d_embed))

    encoder_block = EncoderBlock(
                                 self_attention = copy(attention),
                                 position_ff = copy(position_ff))
    decoder_block = DecoderBlock(
                                 self_attention = copy(attention),
                                 cross_attention = copy(attention),
                                 position_ff = copy(position_ff))

    encoder = Encoder(
                      encoder_block = encoder_block,
                      n_layer = n_layer)
    decoder = Decoder(
                      decoder_block = decoder_block,
                      n_layer = n_layer)
    generator = nn.Linear(d_model, tgt_vocab_size)

    model = Transformer(
                        src_embed = src_embed,
                        tgt_embed = tgt_embed,
                        encoder = encoder,
                        decoder = decoder,
                        generator = generator).to(device)
    model.device = device

    return model